In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import geodesic
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, concatenate, Activation, dot, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import datetime as dt

def load_data(target_lat, target_lon):
    data = pd.read_csv('../datasets/all_month.csv')
    data.dropna(inplace=True)
    data['time'] = pd.to_datetime(data['time']).astype(int) / 1e9
    
    scaler = MinMaxScaler()
    data[['time', 'mag', 'depth']] = scaler.fit_transform(data[['time', 'mag', 'depth']].astype(float))
    data['distance'] = data.apply(lambda row: geodesic((row['latitude'], row['longitude']), (target_lat, target_lon)).kilometers, axis=1)
    
    data = data[data['distance'] <= 500]
    
    return data

def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length].astype(np.float32))
    return np.array(sequences)

def build_model(data, seq_length):
    input_sequences = create_sequences(data[['latitude', 'longitude', 'time', 'mag', 'depth']].values, seq_length)
    target_sequences = create_sequences(data['distance'].values.reshape(-1, 1), seq_length)
    
    inputs = Input(shape=(seq_length, 5))

    lstm_out = LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(inputs)
    lstm_out = LSTM(64, return_sequences=True, dropout=0.1)(lstm_out)

    attention_scores = dot([lstm_out, lstm_out], axes=[2, 2])
    attention_scores = Activation('softmax')(attention_scores)

    context_vector = dot([attention_scores, lstm_out], axes=[2,1])
    context_lstm = concatenate([context_vector, lstm_out])

    output = TimeDistributed(Dense(1))(context_lstm)

    model = Model(inputs=[inputs], outputs=output)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mean_squared_logarithmic_error')
    
    X_train, X_test, y_train, y_test = train_test_split(input_sequences, target_sequences, test_size=0.2, random_state=42)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
    
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['validation_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()
    
    return model

In [ ]:
def predict_distance(model, data, seq_length, target_lat, target_lon):
    data['time'] = pd.to_datetime(data['time']).astype(int) / 1e9
    scaler = MinMaxScaler()
    data[['time', 'mag', 'depth']] = scaler.fit_transform(data[['time', 'mag', 'depth']].astype(float))
    input_data = data[['latitude', 'longitude', 'time', 'mag', 'depth']].values[-seq_length:].reshape(1, seq_length, 5)
    
    predicted_distance = model.predict(input_data)
    
    return predicted_distance

In [ ]:
in_lat = 31.734
in_lon = -118.25
data = load_data(in_lat, in_lon)
model = build_model(data, seq_length=50)

In [ ]:
predicted_distances = predict_distance(model, data, 50, in_lat, in_lon)
expected_distance = predicted_distances[0][-1][0]
print(f"Predicted Distance: {expected_distance} km")

In [ ]:
import folium

map_center = [in_lat, in_lon]
earthquake_map = folium.Map(location=map_center, zoom_start=6)

folium.Circle(
    location=map_center,
    radius=expected_distance * 1000,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.2
).add_to(earthquake_map)

earthquake_map
